# Projet 5A ENR

## F&F

## Apprendre la stratégie journalière de stockage
### Mise en place d'un réseau de neurone

### Objectifs : 
- Estimer le coût d’un mix énergétique de manière rapide en limitant le temps de calcul de l'optimisation du stockage qui se fait aujourd’hui heure par heure. 

L’entrée du réseau peut être par exemple :

1) L’état des stocks des différents moyens de stockage (en valeur absolue ou en pourcentage) à 0h

2) La production nette de la journée (production totale de la journée - consommation totale)

3) le jour de l’année (entre 1 et 365)

Et les sorties seraient : l’état des stocks en fin journée des différents moyens de stockages et le cout de stockage de la journée.
Ceci n’st qu’une proposition. On peut aussi mettre en entrée, la variance de la production journalière qui peut avoir un rôle.

In [1]:
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Prétraitement des données

- il faut genérer les données avec les modèles 

- Quel mix choisir ? Le choisir et l'importer ici =) 

- Le cout de stockage n'est t'il pas aussi dépendant du profil de consommation ? (estimation ADEME, Negawhatt, RTE ?) réponse : le profil de consommation intervient dans la production de la journée nette (qui n'est pas un partie logique)

- Comment on fait pour connaître la production de la journée idem pour la consommation ? Ducoup c'est online ? Quelle idée derrière la variance de la production journalière (+ de variance plus de stockage ?) ? 
- C'est pas un peu con de connaître exactement la production de la journée ? parceque ducoup le stock prédis ca va être sensiblement stock d'entree + prod de la journée : un peu étrange selon Flavie


- produire les données d'apprentissage contenant

    - 1 jour de l'année entre 1 et 365
    - état des stocks à h0
    - production de la journée
    - variance production journalière.
    - variance consomation journalière utile ? (peut être bien que oui aussi)





In [3]:
simu_opti_ademe = pd.read_csv("Project_Renewables_2050_Stage_Amandine/simulations/optimal_simulations/simu_opti_ademe/simu_opti_ademe_hourly_generation.csv")
simu_opti_rte = pd.read_csv("Project_Renewables_2050_Stage_Amandine/simulations/optimal_simulations/simu_opti_rte/simu_opti_rte_hourly_generation.csv")
simu_opti_negawatt = pd.read_csv("Project_Renewables_2050_Stage_Amandine/simulations/optimal_simulations/simu_opti_negawatt/simu_opti_negawatt_hourly_generation.csv")

In [4]:
simu_opti_ademe.drop(["Electricity demand","hour","reserve phs","reserve phs","reserve battery","reserve lake","Storage phs","Storage battery","Storage methanation"],inplace = True, axis =1)
#simu_opti_ademe.drop("hour",inplace = True, axis =1)

NameError: name 'simu_opti_ademe' is not defined

In [5]:
simu_opti_ademe

NameError: name 'simu_opti_ademe' is not defined

In [ ]:
#transforming Data from hour to day
#sum over generation
#hour 24 for stored

In [ ]:
A =simu_opti_ademe[["offshore","onshore","pv","river","lake","biogas","phs","battery","methanation"]].groupby(np.arange(len(simu_opti_ademe))//24).sum()
A

In [ ]:
B = simu_opti_ademe[["Stored phs","Stored battery","Stored methanation"]][::24]
B.reset_index(inplace = True)

In [ ]:
B

In [ ]:
Inputs =[A,B]
Inputs =pd.concat(Inputs,axis =1)

In [ ]:
Inputs

In [ ]:
Inputs["days"]= Inputs.index.values +1
Inputs

In [ ]:
# ajouter le coût du storage
# overnight cost $/kW
# CAPEX $/kWh
# Annuity $/kW/year > pas lié au stockage
# Fixed O&M $/MWh
# Storage annuity $/kWh/year

In [ ]:
stor = ["phs","battery","methanation"]
tec = ["offshore","onshore","pv","river","lake","biogas","phs","battery","methanation"]

# Annualized power capex cost in M€/GW/year
capex = pd.read_csv("inputs/annuities.csv", index_col=0, squeeze=True, header=None)
# Existing capacities of the technologies by December 2017 in GW
capa_ex = pd.read_csv("inputs/existing_capas.csv", index_col=0, squeeze=True, header=None)
# Annualized energy capex cost of storage technologies in M€/GWh/year
capex_en = pd.read_csv("inputs/str_annuities.csv", index_col=0, squeeze=True, header=None)
# Annualized fixed operation and maintenance costs M€/GW/year
fOM = pd.read_csv("inputs/fO&M.csv", index_col=0, squeeze=True, header=None)
# Variable operation and maintenance costs in M€/GWh
vOM = pd.read_csv("inputs/vO&M.csv", index_col=0, squeeze=True, header=None)



# variables optimisés par les 5A 2021
s_capex = [0,0,84.16086]
s_capex = pd.Series(s_capex, index = stor)
# charging related fOM of storage in M€/GW/year
s_opex = [0,0,59.25]
s_opex = pd.Series(s_opex, index = stor)



In [ ]:
vOM = vOM.rename(index={"Onshore": "onshore", "PV":"pv", "PHS":"phs", "Battery":"battery"})
fOM = fOM.rename(index={"Offshore": "offshore", "Onshore": "onshore", "PV":"pv", "PHS":"phs", "Battery":"battery"})
capex = capex.rename(index={"Offshore": "offshore", "Onshore": "onshore", "PV":"pv", "PHS":"phs", "Battery":"battery"})

In [ ]:
tec = ["offshore","onshore","pv","river","lake","biogas","phs","battery","methanation"]
Q = pd.Series([12.64, 79.73, 121.98, 7.5, 13 ,32.93, 9.3, 20.08, 32.93], index = tec)
volume= pd.Series([180 ,74.14, 12499.09], index = stor) #volume du mix optimal 
S = pd.Series([9.3,20.08, 7.66], index = stor)
gene = Inputs[["phs","battery","methanation"]]

In [ ]:
gene

In [ ]:
DAY_FIXED_COST = (sum((Q[tec] - capa_ex[tec]) * capex[tec] for tec in stor) \
+ sum(volume[storage_tecs] * capex_en[storage_tecs] for storage_tecs in stor)\
+ sum(Q[tec] * fOM[tec] for tec in stor)\
+ sum(S[storage_tecs] * (s_opex[storage_tecs] + s_capex[storage_tecs]) for storage_tecs in stor)\
)/(365*1000)

In [ ]:
DAY_VAR_COST = sum((gene[tec] * vOM[tec]) for tec in stor)/1000

In [ ]:
COST = DAY_VAR_COST +DAY_FIXED_COST

In [ ]:
Inputs["Stored phs"] = Inputs["Stored phs"]/volume[0]*100
Inputs["Stored battery"] = Inputs["Stored battery"]/volume[1]*100
Inputs["Stored methanation"] = Inputs["Stored methanation"]/volume[2]*100

In [ ]:
Inputs

In [ ]:
#créer les Xtest, Ytest, Xtrain, Ytrain

In [ ]:
Inputs["cost"]= COST

In [ ]:
# vu sur la cohérence des coûts
np.sum(Inputs["cost"]) #La somme des couts est la même que dans le rapport de sebastien et elisa scénario 1 = optimal

In [ ]:
Inputs["cost"]= COST * 10000

In [ ]:
X = Inputs[:-1]
X.drop(["cost"],inplace = True, axis =1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
XSc= scaler.transform(X)

XSc = pd.DataFrame(XSc, index =X.index, columns = X.columns)

In [ ]:
Y = Inputs[1:]
Y = Y[["Stored phs","Stored battery","Stored methanation","cost"]]
#Y["cost"]=COST[1:]
Y

In [134]:
from sklearn.model_selection import train_test_split 
X_train,X_test,Y_train,Y_test=train_test_split(XSc,Y,test_size=0.25,random_state=11)

## Model de réseau de neurones

Il s'agit d'un truc au hasard d'internet à affiner 
d'internet : 

'Pour la couche Dropout, on a diminué de 30% le nombre des données d’entrée afin d’éviter le phénomène du overfitting. La graine prend une valeur de 2 pour avoir des résultats plus reproductibles.' from https://www.cours-gratuit.com/tutoriel-python/tutoriel-python-matriser-les-rseaux-de-neurones-avec-keras

In [135]:
model = keras.Sequential([
layers.Dense(64, activation = 'relu', input_shape = [X_train.shape[1]]),
layers.Dropout(0.3, seed = 2),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(4)])

### Liste des optimizer à choisir

class Adadelta: Optimizer that implements the Adadelta algorithm.

class Adagrad: Optimizer that implements the Adagrad algorithm.

class Adam: Optimizer that implements the Adam algorithm.

class Adamax: Optimizer that implements the Adamax algorithm.

class Ftrl: Optimizer that implements the FTRL algorithm.

class Nadam: Optimizer that implements the NAdam algorithm.

class Optimizer: Base class for Keras optimizers.

class RMSprop: Optimizer that implements the RMSprop algorithm.

class SGD: Gradient descent (with momentum) optimizer.

## Liste des losses à choisir

class BinaryCrossentropy: Computes the cross-entropy loss between true labels and predicted labels.

class BinaryFocalCrossentropy: Computes the focal cross-entropy loss between true labels and predictions.

class CategoricalCrossentropy: Computes the crossentropy loss between the labels and predictions.

class CategoricalHinge: Computes the categorical hinge loss between y_true and y_pred.

class CosineSimilarity: Computes the cosine similarity between labels and predictions.

class Hinge: Computes the hinge loss between y_true and y_pred.

class Huber: Computes the Huber loss between y_true and y_pred.

class KLDivergence: Computes Kullback-Leibler divergence loss between y_true and y_pred.

class LogCosh: Computes the logarithm of the hyperbolic cosine of the prediction error.

class Loss: Loss base class.

class MeanAbsoluteError: Computes the mean of absolute difference between labels and predictions.

class MeanAbsolutePercentageError: Computes the mean absolute percentage error between y_true and y_pred.

class MeanSquaredError: Computes the mean of squares of errors between labels and predictions.

class MeanSquaredLogarithmicError: Computes the mean squared logarithmic error between y_true and y_pred.

class Poisson: Computes the Poisson loss between y_true and y_pred.

class Reduction: Types of loss reduction.

class SparseCategoricalCrossentropy: Computes the crossentropy loss between the labels and predictions.

class SquaredHinge: Computes the squared hinge loss between y_true and y_pred.

## Choisir une metrics

In [136]:
optimiseur = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
model.compile(loss = tf.keras.losses.MeanSquaredError(),
optimizer = optimiseur,
metrics = ['accuracy'])

training = model.fit (X_train, Y_train, epochs = 70, validation_split = 0.2)

Epoch 1/70
7/7 [==============================] - 2s 43ms/step - loss: 2777.3225 - accuracy: 0.4954 - val_loss: 2395.7363 - val_accuracy: 0.5818
Epoch 2/70
7/7 [==============================] - 0s 8ms/step - loss: 2207.3845 - accuracy: 0.5321 - val_loss: 1214.2571 - val_accuracy: 0.5818
Epoch 3/70
7/7 [==============================] - 0s 7ms/step - loss: 1035.3085 - accuracy: 0.5321 - val_loss: 664.5140 - val_accuracy: 0.5818
Epoch 4/70
7/7 [==============================] - 0s 10ms/step - loss: 769.0532 - accuracy: 0.5321 - val_loss: 592.0479 - val_accuracy: 0.5818
Epoch 5/70
7/7 [==============================] - 0s 11ms/step - loss: 740.2855 - accuracy: 0.5321 - val_loss: 548.4908 - val_accuracy: 0.5818
Epoch 6/70
7/7 [==============================] - 0s 9ms/step - loss: 689.4142 - accuracy: 0.5321 - val_loss: 522.1774 - val_accuracy: 0.5818
Epoch 7/70
7/7 [==============================] - 0s 10ms/step - loss: 660.7994 - accuracy: 0.5321 - val_loss: 525.3721 - val_accuracy: 0.58

7/7 [==============================] - 0s 10ms/step - loss: 240.6946 - accuracy: 0.7844 - val_loss: 208.4170 - val_accuracy: 0.8364
Epoch 59/70
7/7 [==============================] - 0s 12ms/step - loss: 247.5766 - accuracy: 0.7752 - val_loss: 217.5688 - val_accuracy: 0.8182
Epoch 60/70
7/7 [==============================] - 0s 11ms/step - loss: 270.6331 - accuracy: 0.8073 - val_loss: 210.0772 - val_accuracy: 0.8364
Epoch 61/70
7/7 [==============================] - 0s 11ms/step - loss: 218.9654 - accuracy: 0.8028 - val_loss: 218.6039 - val_accuracy: 0.8182
Epoch 62/70
7/7 [==============================] - 0s 7ms/step - loss: 254.7353 - accuracy: 0.7982 - val_loss: 229.8176 - val_accuracy: 0.8182
Epoch 63/70
7/7 [==============================] - 0s 11ms/step - loss: 228.6841 - accuracy: 0.8028 - val_loss: 212.5087 - val_accuracy: 0.8364
Epoch 64/70
7/7 [==============================] - 0s 10ms/step - loss: 231.5490 - accuracy: 0.7890 - val_loss: 219.3495 - val_accuracy: 0.8364
Epoch

In [137]:
score =model.evaluate(X_test,Y_test)

3/3 [==============================] - 0s 5ms/step - loss: 196.5165 - accuracy: 0.8791


In [146]:
res = pd.DataFrame(model.predict(X_test))
res

3/3 [==============================] - 0s 4ms/step


,0,1,2,3
0,19.994944,8.464035,75.997765,72.722420
1,12.643332,6.057559,70.499290,71.414810
2,34.165474,12.230449,60.669357,69.667809
3,11.397560,6.680677,79.242203,70.270920
4,0.073449,0.976953,36.013611,65.185677
...,...,...,...,...
86,14.094468,12.492035,79.917465,66.589195
87,27.710577,10.485025,67.309471,71.267685
88,35.994289,20.724720,67.480484,73.290337
89,11.286255,7.199666,63.672096,67.956009


In [148]:
np.sum(res[3])

6467.169

In [139]:
Y_test

,Stored phs,Stored battery,Stored methanation,cost
261,17.222222,0.000000,72.771778,68.52267
229,3.983333,0.000000,67.231294,68.20095
155,37.438889,24.035608,56.135767,67.39917
318,13.194444,0.000000,79.056875,67.04364
140,0.000000,0.000000,44.062168,67.04364
...,...,...,...,...
364,0.000000,0.000000,64.281880,67.04364
165,27.477778,4.504991,63.785924,68.58945
219,0.000000,0.000000,58.601626,68.16063
196,0.000000,0.000000,62.050917,67.04364


In [140]:
np.sum(Y_test['cost'])

6234.226515729628